In [47]:
# -*- coding:utf-8 -*- 

In [281]:
import pandas as pd
import traitlets
from IPython.display import display,update_display
import ipywidgets as widgets
import bqplot as bp
import random

In [282]:
import sys 
sys.path.append('libs/')
from table_v3 import TableChart

In [283]:
class EtlCommand(traitlets.HasTraits):
    
    _df = pd.DataFrame()
    #_data = traitlets.Dict()
    #_header = traitlets.Dict()
    _table = None
    _show = False
    _GUI = None
 
    def __init__(self,df):
        self._df = df
        self._data = df.values.tolist()
        self._header = df.columns.values.tolist()
        self._table = TableChart(self._header,self._data)
    
    def drop(self,**kwargs):
        col_value = kwargs.get('col')
        print ('drop:'+ col_value)
        if col_value and col_value in self._header:
            self._df.drop(col_value,axis=1,inplace=True)
            self.update()
        else:
            raise Exception('col is not valid, must be in list[{}]'.format(self._df.columns.values.tolist()))
    
    def update(self):
        #self._data = df.values.tolist()
        #self._header = df.columns.values.tolist()
        if self._show:
            self._table.model.data = self._df.values.tolist()
            self._table.model.header = self._df.columns.values.tolist()
            kwargs={'id':random.random(),'operation':'drop','col':df.columns.values.tolist()}
            #update_display(self._updatehandle,display_id='commandGUI')
            self._GUI.cmdargs = kwargs
    
    def showTable(self,enable=True):
        if isinstance(enable,bool):
            self._show = enable
        else:
            raise Exception('display must be bool value')
        if self._show:
            self.show_handle = display(self._table)
        else:
            self._table.close()
            self._show = False
     
    def showGUI(self):
        self._GUI = EtlGUI(self,{'operation':'drop','col':df.columns.values.tolist()})
        self._updatehandle = display(self._GUI,display_id='commandGUI')


In [425]:
class EtlGUI(widgets.Box):
    
    _cmds = None
    _args = None
    _argsdict = traitlets.Dict()
    
    def __init__(self,cmds,kwargs=None):
        super(EtlGUI,self).__init__()
        self._cmds = cmds
        self._argsdict = kwargs 
        self._args = self.initArgs()
        self._btn_ok = widgets.Button(description=u'确认',layout=widgets.Layout(width='10%',))
        self.children = [self._lb_operation,self._args,self._btn_ok]
        self.layout = widgets.Layout(width="100%",display='flex-flow',flex_flow='row', justify_content=u'flex-start')
        self._btn_ok.on_click(self.execute)
        
    def execute(self,a):
        cmd_label = self._lb_operation.value
        action_handler = getattr(self._cmds,cmd_label)
        if not action_handler:
            raise Exception('label:{} is not a valid ETL command '.format(cmd_label))
        args = {}
        for arg in self._args.children:
            arg_name = arg.children[0].value
            arg_value = arg.children[1].value
            args[arg_name] = arg_value
        print(args)
        action_handler(**args)
    
    @property
    def cmdargs(self):
        return self._argsdict
    
    @cmdargs.setter
    def cmdargs(self,kwargs=None):
        self._argsdict = kwargs
    
    
    
    @traitlets.observe('_argsdict')
    def updateArgs(self,change=None):
        
        if not self._args:
            return 
        for arg in self._args.children:
            arg_name = arg.children[0].value
            if arg_name in self._argsdict:
                arg.children[1].options =  self._argsdict[arg_name]
                arg.children[1].value =  None

        
    def initArgs(self):
        kwargs = self._argsdict
        print(kwargs)
        if 'id' in kwargs:
            del kwargs['id']
        width_auto =  '{}%'.format(100 / len(kwargs))
        input_args = []

        for arg in kwargs:
            value = kwargs[arg]
            if arg ==u'operation':
                self._lb_operation = widgets.Label(value,layout=widgets.Layout(width='10%'))
                continue
            if isinstance(value,int):
                widget_arg = widgets.IntText(value)
            elif isinstance(value,list):
                widget_arg = widgets.Dropdown(options=value,value=None)
                if value =='col':
                    widget_arg.observe(self.fn,'value')
                    
            elif isinstance(value,str):
                widget_arg = widgets.Text(value)
            group_arg = widgets.HBox(layout=widgets.Layout(width=width_auto))
            width_arg_label = '{}px'.format(len(arg)*16)
            group_arg.children = [widgets.Label(arg,layout=widgets.Layout(width=width_arg_label)),widget_arg]
            input_args.append(group_arg)
        
        arg_hbox = widgets.HBox(layout=widgets.Layout(width='80%'))
        arg_hbox.children = input_args
        return arg_hbox
    
    def fn(change):
        value=change['new']
        

class ColWidget(widgets.Box):
    
    value = traitlets.Unicode()
    def __init__(self,label='Col',options=None,placehold='please choose a column'):
        super(ColWidget,self).__init__()
        
        self.layout = widgets.Layout(width="100%",display='flex-flow',flex_flow='row', justify_content=u'flex-start')
        self.placehold = placehold
        labels = widgets.Label(label,layout =widgets.Layout(width='64px') )
        if not(isinstance(options,list)):
            raise Exception('options must be a list')
        self.inputs = widgets.Dropdown(options =[self.placehold]+options,value=self.placehold )
        self.children = [labels,self.inputs]
        self.inputs.observe(self.eventColChoose,'value')   
        
    def eventColChoose(self,change):
        if change['new'] != self.placehold:
            self.value = change['new']

class FilterWidget(widgets.Box):
    value = traitlets.Unicode()
    apply = traitlets.Bool(False)
    
    def __init__(self,options=None,placehold='please choose a column'):
        super(FilterWidget,self).__init__()
        self.layout = layout=widgets.Layout(display='flex-flow',flex_flow='column')
        cmdSection = widgets.Box(layout=widgets.Layout(display='flex-flow',flex_flow='row', justify_content=u'flex-start'))
        self.wColWidget = ColWidget(options=df.columns.values.tolist(),placehold=u'',label='请选择列')
        self.btnAdd = widgets.Button(description=u'增加条件')
        self.btnApply = widgets.Button(description=u'应用筛选')
        self.btnCancel = widgets.Button(description=u'取消筛选')
        cmdSection.children=[self.wColWidget,self.btnAdd,self.btnApply,self.btnCancel]
        self.children = [cmdSection]
        self.btnAdd.on_click(self.andFilter,False)
        self.btnApply.on_click(self.filterApply,False)
        self.btnCancel.on_click(self.filterCancel,False)
    
    def andFilter(self,a):
        partCol = widgets.Label(self.wColWidget.value,layout=widgets.Layout(width='100px'))
        partOp = widgets.Dropdown(options=['==','>','>=','<','<=','<>','in','between'],layout=widgets.Layout(width='100px'))
        partInput = widgets.Text(placehold=u'请输入筛选范围')
        partAnd = widgets.Label('and',layout=widgets.Layout(width='100px'))
        addCondition = widgets.Box(layout=widgets.Layout(display='flex-flow',flex_flow='row', justify_content=u'flex-start'))
        addCondition.children = [partCol,partOp,partInput,partAnd ]
        self.children = [c for c in self.children] + [addCondition]
    
    def filterApply(self,a):
        cond = []
        for b in f.children[1:]:
            field = b.children[0].value
            op = b.children[1].value
            value = b.children[2].value
            cond.append("({} {} {})".format(field , op ,value))
        self.value = ' and '.join(cond)
        self.apply = True
        
    def filterCancel(self,a):
        self.apply = False    
        
        

In [521]:

f =FilterWidget(options=df.columns.values.tolist(),placehold=u'请选择列')
f

A Jupyter Widget

(petalLength == 33) and (species in ['a','b','c']) and (species <> 'd')


In [522]:
f.value

"(petalLength == 33) and (species in ['a','b','c'])"

In [523]:
def fn(change):
    print(change['new'])
f.observe(fn,'value')   

In [494]:
df = pd.read_csv('../data/iris.csv')

In [279]:
cmd = EtlCommand(df)
cmd.showTable()

A Jupyter Widget

In [280]:
cmd.showGUI()

{'operation': 'drop', 'col': ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth', 'species']}


A Jupyter Widget

{'col': 'sepalWidth'}
{'col': 'sepalLength'}


In [245]:
cmd._df.columns

Index(['petalWidth', 'sepalLength', 'sepalWidth', 'species'], dtype='object')

In [244]:
cmd._df.head()

,petalWidth,sepalLength,sepalWidth,species
0,0.2,5.1,3.5,setosa
1,0.2,4.9,3.0,setosa
2,0.2,4.7,3.2,setosa
3,0.2,4.6,3.1,setosa
4,0.2,5.0,3.6,setosa


In [193]:
print(cmd._GUI._args)

In [170]:
random.random()

0.13135690348335494

In [228]:
getattr(cmd._GUI,'__init__')

<method-wrapper '__init__' of NoneType object at 0x7f838c14bd60>

In [229]:
dir(cmd._GUI)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [275]:
d = widgets.Dropdown(options=[1,2,3,4])


def p(y):
    print(y)
    
d.observe(p,'value')


In [276]:
d

A Jupyter Widget

{'name': 'value', 'old': 1, 'new': 3, 'owner': Dropdown(index=2, options=(1, 2, 3, 4), value=3), 'type': 'change'}
{'name': 'value', 'old': 3, 'new': 4, 'owner': Dropdown(index=3, options=(1, 2, 3, 4), value=4), 'type': 'change'}
